In [1]:
import sys
sys.path.append('/home/daniel/gitrepos/vision/references/detection')
import torch
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.utils.data
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm

import collections
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
csv_dir = "/hdd/open-images/csvs"
train_df = pd.read_csv(f"{csv_dir}/train-annotations-bbox.csv")

In [3]:
# label_encoder = LabelEncoder()
# train_df["LabelEncoded"] = label_encoder.fit_transform(train_df["LabelName"])
# print("Finished encoding labels")
# collapsed_df = train_df.groupby('ImageID', as_index=False).agg(lambda x: " ".join(x.astype(str)))

In [4]:
# print("Original length: ", len(train_df))
# print("Collapsed length: ", len(collapsed_df))
# collapsed_df.to_csv(f"{csv_dir}/train-annotations-bbox-collapsed.csv", index=False)

In [5]:
# torch.save(label_encoder,"label_encoder.bin")
label_encoder = torch.load("label_encoder.bin")

In [7]:
def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

class OpenDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, df_path, height, width, img_transforms=None):
        self.transforms = img_transforms
        self.image_dir = image_dir
        self.df = pd.read_csv(df_path)
#         self.df = self.df[:1000]
        self.height = height
        self.width = width
        self.image_info = collections.defaultdict(dict)
        
        # Filling up image_info is left as an exercise to the reader
        
        counter = 0
        for index, row in tqdm(self.df.iterrows(), total=len(self.df)):
            image_id = row["ImageID"]
            image_path = os.path.join(self.image_dir, image_id)

            if os.path.exists(image_path + '.jpg'):
                self.image_info[counter]["image_id"] = image_id
                self.image_info[counter]["image_path"] = image_path
                self.image_info[counter]["XMin"] = row["XMin"]
                self.image_info[counter]["YMin"] = row["YMin"]
                self.image_info[counter]["XMax"] = row["XMax"]
                self.image_info[counter]["YMax"] = row["YMax"]
                self.image_info[counter]["labels"] = row["LabelEncoded"]
                counter += 1

    def __getitem__(self, idx):
        # load images ad masks
        img_path = self.image_info[idx]["image_path"] + ".jpg"
        img = Image.open(img_path).convert("RGB")
        img = img.resize((self.width, self.height), resample=Image.BILINEAR)
        
        # processing part and extraction of boxes is left as an exercise to the reader
        # get bounding box coordinates for each mask         
        num_objs = len(self.image_info[idx]["labels"].split())
        
        boxes = []
        xmins = self.image_info[idx]["XMin"].split()
        ymins = self.image_info[idx]["YMin"].split()
        xmaxs = self.image_info[idx]["XMax"].split()
        ymaxs = self.image_info[idx]["YMax"].split()
        
        assert len(xmins) == len(ymins) == len(xmaxs) == len(ymaxs) == num_objs
        
        for i in range(num_objs):
            xmin = float(xmins[i])
            xmax = float(xmaxs[i])
            ymin = float(ymins[i])
            ymax = float(ymaxs[i])
            boxes.append([xmin, ymin, xmax, ymax])
                                                
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor([int(x) for x in self.image_info[idx]["labels"].split()])

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
                                                              
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        else:
            img = transforms.ToTensor()(img)

#         print(img)
#         print(target)
        return img, target

    def __len__(self):
        return len(self.image_info)
    

In [8]:
csv_dir = "/hdd/open-images/csvs"
train_csv = pd.read_csv(f"{csv_dir}/train-annotations-bbox.csv")
label_counts = train_csv["LabelName"].value_counts()
num_classes = len(label_counts)
# num_greater_than_50 = len(label_counts[label_counts >= 50])
# print("Number of classes with at least 50 examples: ", num_greater_than_50)
# num_classes = num_greater_than_50
print(num_classes)

599


In [9]:
device = torch.device('cuda:1')

train_images_dir = "data/unzipped/all_train"
train_csv_file = "data/csvs/train-annotations-bbox-collapsed.csv"
dataset_train = OpenDataset(train_images_dir, train_csv_file, 128, 128, img_transforms=None)
print(len(dataset_train))

100%|██████████| 1743042/1743042 [02:41<00:00, 10818.73it/s]

1543042


In [ ]:
model_ft = get_instance_segmentation_model(num_classes)
model_ft.to(device)

data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=4, shuffle=True, num_workers=8,
    collate_fn=utils.collate_fn)

In [ ]:
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)
num_epochs = 8
for epoch in range(num_epochs):
    train_one_epoch(model_ft, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()

torch.save(model_ft.state_dict(), "model.bin")

Epoch: [0]  [     0/385761]  eta: 9 days, 21:21:32  lr: 0.000010  loss: 11.5014 (11.5014)  loss_classifier: 6.3867 (6.3867)  loss_box_reg: 0.0084 (0.0084)  loss_objectness: 3.8171 (3.8171)  loss_rpn_box_reg: 1.2891 (1.2891)  time: 2.2151  data: 1.3998  max mem: 0
Epoch: [0]  [    10/385761]  eta: 2 days, 17:14:36  lr: 0.000060  loss: 10.1931 (10.2585)  loss_classifier: 6.3118 (6.2788)  loss_box_reg: 0.0070 (0.0065)  loss_objectness: 2.9154 (3.0056)  loss_rpn_box_reg: 0.9235 (0.9676)  time: 0.6089  data: 0.1328  max mem: 0
Epoch: [0]  [    20/385761]  eta: 2 days, 8:30:47  lr: 0.000110  loss: 7.8509 (8.1741)  loss_classifier: 5.8992 (5.6127)  loss_box_reg: 0.0064 (0.0057)  loss_objectness: 1.3857 (1.8021)  loss_rpn_box_reg: 0.7739 (0.7535)  time: 0.4430  data: 0.0040  max mem: 0
Epoch: [0]  [    30/385761]  eta: 2 days, 5:24:13  lr: 0.000160  loss: 3.5786 (6.0417)  loss_classifier: 2.8640 (4.1238)  loss_box_reg: 0.0064 (0.0084)  loss_objectness: 0.1691 (1.2729)  loss_rpn_box_reg: 0.4103

Epoch: [0]  [   320/385761]  eta: 1 day, 23:52:03  lr: 0.001608  loss: 0.3745 (1.0415)  loss_classifier: 0.1141 (0.5871)  loss_box_reg: 0.0206 (0.0167)  loss_objectness: 0.0021 (0.1355)  loss_rpn_box_reg: 0.2207 (0.3022)  time: 0.4437  data: 0.0043  max mem: 0
Epoch: [0]  [   330/385761]  eta: 1 day, 23:51:09  lr: 0.001658  loss: 0.3670 (1.0223)  loss_classifier: 0.1373 (0.5742)  loss_box_reg: 0.0206 (0.0167)  loss_objectness: 0.0024 (0.1315)  loss_rpn_box_reg: 0.2028 (0.2999)  time: 0.4442  data: 0.0039  max mem: 0
Epoch: [0]  [   340/385761]  eta: 1 day, 23:50:19  lr: 0.001708  loss: 0.3761 (1.0046)  loss_classifier: 0.1577 (0.5627)  loss_box_reg: 0.0183 (0.0169)  loss_objectness: 0.0024 (0.1277)  loss_rpn_box_reg: 0.2275 (0.2972)  time: 0.4429  data: 0.0030  max mem: 0
Epoch: [0]  [   350/385761]  eta: 1 day, 23:49:27  lr: 0.001758  loss: 0.4160 (0.9878)  loss_classifier: 0.1632 (0.5508)  loss_box_reg: 0.0216 (0.0170)  loss_objectness: 0.0017 (0.1242)  loss_rpn_box_reg: 0.2188 (0.29

Epoch: [0]  [   640/385761]  eta: 1 day, 23:41:59  lr: 0.003207  loss: 0.3980 (0.7319)  loss_classifier: 0.1111 (0.3632)  loss_box_reg: 0.0146 (0.0170)  loss_objectness: 0.0028 (0.0693)  loss_rpn_box_reg: 0.2422 (0.2824)  time: 0.4450  data: 0.0038  max mem: 0
Epoch: [0]  [   650/385761]  eta: 1 day, 23:42:00  lr: 0.003257  loss: 0.3683 (0.7278)  loss_classifier: 0.1157 (0.3602)  loss_box_reg: 0.0164 (0.0170)  loss_objectness: 0.0030 (0.0683)  loss_rpn_box_reg: 0.2333 (0.2823)  time: 0.4452  data: 0.0038  max mem: 0
Epoch: [0]  [   660/385761]  eta: 1 day, 23:41:46  lr: 0.003307  loss: 0.3700 (0.7227)  loss_classifier: 0.1052 (0.3563)  loss_box_reg: 0.0159 (0.0169)  loss_objectness: 0.0024 (0.0673)  loss_rpn_box_reg: 0.2333 (0.2822)  time: 0.4455  data: 0.0038  max mem: 0
Epoch: [0]  [   670/385761]  eta: 1 day, 23:41:38  lr: 0.003357  loss: 0.3396 (0.7177)  loss_classifier: 0.1014 (0.3526)  loss_box_reg: 0.0108 (0.0169)  loss_objectness: 0.0019 (0.0663)  loss_rpn_box_reg: 0.2327 (0.28

Epoch: [0]  [   960/385761]  eta: 1 day, 23:38:47  lr: 0.004805  loss: 0.3824 (0.6248)  loss_classifier: 0.1017 (0.2860)  loss_box_reg: 0.0160 (0.0168)  loss_objectness: 0.0016 (0.0470)  loss_rpn_box_reg: 0.2611 (0.2750)  time: 0.4457  data: 0.0038  max mem: 0
Epoch: [0]  [   970/385761]  eta: 1 day, 23:38:38  lr: 0.004855  loss: 0.3806 (0.6222)  loss_classifier: 0.0999 (0.2841)  loss_box_reg: 0.0140 (0.0168)  loss_objectness: 0.0015 (0.0465)  loss_rpn_box_reg: 0.2611 (0.2747)  time: 0.4443  data: 0.0037  max mem: 0
Epoch: [0]  [   980/385761]  eta: 1 day, 23:38:33  lr: 0.004905  loss: 0.3423 (0.6201)  loss_classifier: 0.1110 (0.2829)  loss_box_reg: 0.0160 (0.0169)  loss_objectness: 0.0013 (0.0461)  loss_rpn_box_reg: 0.2206 (0.2743)  time: 0.4451  data: 0.0036  max mem: 0
Epoch: [0]  [   990/385761]  eta: 1 day, 23:38:23  lr: 0.004955  loss: 0.3273 (0.6171)  loss_classifier: 0.0999 (0.2810)  loss_box_reg: 0.0167 (0.0168)  loss_objectness: 0.0009 (0.0456)  loss_rpn_box_reg: 0.1975 (0.27

Epoch: [0]  [  1280/385761]  eta: 1 day, 23:35:23  lr: 0.005000  loss: 0.3647 (0.5678)  loss_classifier: 0.1173 (0.2471)  loss_box_reg: 0.0129 (0.0168)  loss_objectness: 0.0015 (0.0359)  loss_rpn_box_reg: 0.2329 (0.2681)  time: 0.4449  data: 0.0036  max mem: 0
Epoch: [0]  [  1290/385761]  eta: 1 day, 23:35:17  lr: 0.005000  loss: 0.4104 (0.5666)  loss_classifier: 0.1177 (0.2461)  loss_box_reg: 0.0140 (0.0168)  loss_objectness: 0.0030 (0.0356)  loss_rpn_box_reg: 0.2622 (0.2681)  time: 0.4450  data: 0.0036  max mem: 0
Epoch: [0]  [  1300/385761]  eta: 1 day, 23:35:07  lr: 0.005000  loss: 0.4199 (0.5657)  loss_classifier: 0.1228 (0.2453)  loss_box_reg: 0.0143 (0.0168)  loss_objectness: 0.0022 (0.0354)  loss_rpn_box_reg: 0.2538 (0.2683)  time: 0.4445  data: 0.0036  max mem: 0
Epoch: [0]  [  1310/385761]  eta: 1 day, 23:35:04  lr: 0.005000  loss: 0.4199 (0.5647)  loss_classifier: 0.1228 (0.2444)  loss_box_reg: 0.0150 (0.0168)  loss_objectness: 0.0017 (0.0351)  loss_rpn_box_reg: 0.2876 (0.26

...

Epoch: [0]  [ 59770/385761]  eta: 1 day, 16:27:30  lr: 0.005000  loss: 0.4032 (0.3555)  loss_classifier: 0.1055 (0.1058)  loss_box_reg: 0.0085 (0.0122)  loss_objectness: 0.0010 (0.0025)  loss_rpn_box_reg: 0.2481 (0.2351)  time: 0.4498  data: 0.0037  max mem: 0
Epoch: [0]  [ 59780/385761]  eta: 1 day, 16:27:25  lr: 0.005000  loss: 0.3260 (0.3555)  loss_classifier: 0.0872 (0.1058)  loss_box_reg: 0.0100 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2249 (0.2351)  time: 0.4490  data: 0.0037  max mem: 0
Epoch: [0]  [ 59790/385761]  eta: 1 day, 16:27:21  lr: 0.005000  loss: 0.3260 (0.3555)  loss_classifier: 0.0770 (0.1058)  loss_box_reg: 0.0124 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2249 (0.2351)  time: 0.4478  data: 0.0037  max mem: 0
Epoch: [0]  [ 59800/385761]  eta: 1 day, 16:27:16  lr: 0.005000  loss: 0.3503 (0.3555)  loss_classifier: 0.0675 (0.1058)  loss_box_reg: 0.0102 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2553 (0.23

Epoch: [0]  [ 60090/385761]  eta: 1 day, 16:25:09  lr: 0.005000  loss: 0.4028 (0.3554)  loss_classifier: 0.1103 (0.1057)  loss_box_reg: 0.0087 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2782 (0.2351)  time: 0.4486  data: 0.0037  max mem: 0
Epoch: [0]  [ 60100/385761]  eta: 1 day, 16:25:05  lr: 0.005000  loss: 0.3583 (0.3554)  loss_classifier: 0.1018 (0.1057)  loss_box_reg: 0.0100 (0.0122)  loss_objectness: 0.0010 (0.0025)  loss_rpn_box_reg: 0.2361 (0.2351)  time: 0.4491  data: 0.0036  max mem: 0
Epoch: [0]  [ 60110/385761]  eta: 1 day, 16:25:00  lr: 0.005000  loss: 0.3492 (0.3554)  loss_classifier: 0.0914 (0.1057)  loss_box_reg: 0.0082 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2181 (0.2351)  time: 0.4490  data: 0.0036  max mem: 0
Epoch: [0]  [ 60120/385761]  eta: 1 day, 16:24:56  lr: 0.005000  loss: 0.3035 (0.3554)  loss_classifier: 0.0913 (0.1057)  loss_box_reg: 0.0085 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.1987 (0.23

Epoch: [0]  [ 60410/385761]  eta: 1 day, 16:22:48  lr: 0.005000  loss: 0.3211 (0.3554)  loss_classifier: 0.0876 (0.1057)  loss_box_reg: 0.0139 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.1972 (0.2351)  time: 0.4483  data: 0.0037  max mem: 0
Epoch: [0]  [ 60420/385761]  eta: 1 day, 16:22:44  lr: 0.005000  loss: 0.3136 (0.3554)  loss_classifier: 0.0927 (0.1057)  loss_box_reg: 0.0102 (0.0122)  loss_objectness: 0.0010 (0.0025)  loss_rpn_box_reg: 0.2052 (0.2351)  time: 0.4491  data: 0.0038  max mem: 0
Epoch: [0]  [ 60430/385761]  eta: 1 day, 16:22:39  lr: 0.005000  loss: 0.3119 (0.3554)  loss_classifier: 0.0920 (0.1057)  loss_box_reg: 0.0094 (0.0122)  loss_objectness: 0.0009 (0.0025)  loss_rpn_box_reg: 0.2062 (0.2351)  time: 0.4489  data: 0.0039  max mem: 0
Epoch: [0]  [ 60440/385761]  eta: 1 day, 16:22:35  lr: 0.005000  loss: 0.3121 (0.3554)  loss_classifier: 0.0867 (0.1057)  loss_box_reg: 0.0109 (0.0122)  loss_objectness: 0.0013 (0.0025)  loss_rpn_box_reg: 0.1864 (0.23

Epoch: [0]  [ 60730/385761]  eta: 1 day, 16:20:27  lr: 0.005000  loss: 0.3522 (0.3553)  loss_classifier: 0.0870 (0.1057)  loss_box_reg: 0.0105 (0.0122)  loss_objectness: 0.0011 (0.0025)  loss_rpn_box_reg: 0.2323 (0.2350)  time: 0.4480  data: 0.0037  max mem: 0
Epoch: [0]  [ 60740/385761]  eta: 1 day, 16:20:23  lr: 0.005000  loss: 0.3273 (0.3553)  loss_classifier: 0.0809 (0.1057)  loss_box_reg: 0.0103 (0.0122)  loss_objectness: 0.0010 (0.0025)  loss_rpn_box_reg: 0.1961 (0.2350)  time: 0.4477  data: 0.0037  max mem: 0
Epoch: [0]  [ 60750/385761]  eta: 1 day, 16:20:18  lr: 0.005000  loss: 0.2889 (0.3553)  loss_classifier: 0.0813 (0.1057)  loss_box_reg: 0.0116 (0.0122)  loss_objectness: 0.0010 (0.0025)  loss_rpn_box_reg: 0.1798 (0.2350)  time: 0.4475  data: 0.0038  max mem: 0
Epoch: [0]  [ 60760/385761]  eta: 1 day, 16:20:14  lr: 0.005000  loss: 0.3133 (0.3553)  loss_classifier: 0.0944 (0.1057)  loss_box_reg: 0.0138 (0.0122)  loss_objectness: 0.0013 (0.0025)  loss_rpn_box_reg: 0.1966 (0.23

Epoch: [0]  [ 61050/385761]  eta: 1 day, 16:18:06  lr: 0.005000  loss: 0.3660 (0.3553)  loss_classifier: 0.0929 (0.1056)  loss_box_reg: 0.0094 (0.0122)  loss_objectness: 0.0008 (0.0024)  loss_rpn_box_reg: 0.2469 (0.2350)  time: 0.4479  data: 0.0037  max mem: 0
Epoch: [0]  [ 61060/385761]  eta: 1 day, 16:18:02  lr: 0.005000  loss: 0.3660 (0.3553)  loss_classifier: 0.0982 (0.1056)  loss_box_reg: 0.0100 (0.0122)  loss_objectness: 0.0008 (0.0024)  loss_rpn_box_reg: 0.2586 (0.2350)  time: 0.4471  data: 0.0037  max mem: 0
Epoch: [0]  [ 61070/385761]  eta: 1 day, 16:17:57  lr: 0.005000  loss: 0.2982 (0.3553)  loss_classifier: 0.0840 (0.1056)  loss_box_reg: 0.0094 (0.0122)  loss_objectness: 0.0008 (0.0024)  loss_rpn_box_reg: 0.2058 (0.2350)  time: 0.4473  data: 0.0037  max mem: 0
Epoch: [0]  [ 61080/385761]  eta: 1 day, 16:17:53  lr: 0.005000  loss: 0.3386 (0.3553)  loss_classifier: 0.0871 (0.1056)  loss_box_reg: 0.0079 (0.0122)  loss_objectness: 0.0009 (0.0024)  loss_rpn_box_reg: 0.2164 (0.23